In [1]:
print("Hello LangChain")

Hello LangChain


In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

#load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY[:10])
print(OPENAI_API_KEY[:5])

gsk_3


In [20]:
# prompt + llm + output 

# prompt: (쉽게 말해) AI에게 하는 질문
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , # AI에게 역할을 주는 것.
     ("user", "{input}") ]
)
print(type(prompt))
print(prompt)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [30]:
prompt_text = prompt.format(input="파이썬은 무엇인가요? 자세하게 설명해주세요")
print(type(prompt_text))
print(prompt_text)

<class 'str'>
System: 당신은 개발자입니다.
Human: 파이썬은 무엇인가요? 자세하게 설명해주세요


In [37]:
#llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125")

# Groq는 ChatGPT와 호환 -> ChatGPT 모델 사용 가능
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성 (모델 가져와서 생성)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct", # 사용 모델: llama-4-scout
    #model="moonshotai/kimi-k2-instruct-0905", # 사용 모델: kimi-k2
    model="openai/gpt-oss-120b", # 사용 모델: gpt-oss-120b
    temperature=0.7
)
print(type(llm)) # 타입: ChatOpenAI
print(llm)

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
client=<openai.resources.chat.completions.completions.Completions object at 0x140293790> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1402ba610> root_client=<openai.OpenAI object at 0x14011d2d0> root_async_client=<openai.AsyncOpenAI object at 0x140292490> model_name='openai/gpt-oss-120b' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [38]:
# 생성한 모델에 질문하기(prompt 입력)
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
응답: ## 파이썬(Python)이란?

파이썬은 **고급(high‑level) 프로그래밍 언어**이며, 1991년 네덜란드의 귀도 반 로섬(Guido van Rossum)이 처음 발표했습니다. 현재는 전 세계 수많은 개발자와 기업이 사용하고 있는 가장 인기 있는 언어 중 하나이며, **읽기 쉬운 문법**, **다양한 라이브러리·프레임워크**, **광범위한 적용 분야**가 특징입니다.

---

## 1. 파이썬의 주요 특징

| 특징 | 설명 |
|------|------|
| **읽기 쉬운 문법** | 코드가 자연어에 가깝고 들여쓰기로 블록을 구분해 가독성이 뛰어납니다. |
| **동적 타이핑** | 변수 선언 시 타입을 명시하지 않아도 런타임에 자동으로 결정됩니다. |
| **인터프리터 언어** | 소스 코드를 바로 실행할 수 있어 빠른 테스트와 프로토타이핑이 가능합니다. |
| **멀티패러다임** | 절차형, 객체지향, 함수형 프로그래밍을 모두 지원합니다. |
| **풍부한 표준 라이브러리** | 파일 I/O, 네트워킹, 웹, 데이터베이스, 암호화 등 2만 개 이상의 모듈 제공. |
| **플랫폼 독립성** | Windows, macOS, Linux 등 거의 모든 운영체제에서 동일하게 동작합니다. |
| **확장성** | C/C++ 로 작성된 모듈을 쉽게 호출하거나, Cython, PyPy 등으로 성능을 향상시킬 수 있습니다. |
| **활발한 커뮤니티** | 전 세계 개발자·연구자들이 만든 수많은 오픈소스 패키지와 튜토리얼이 존재합니다. |

---

## 2. 파이썬의 기본 문법

### 2‑1. 변수와 자료형
```python
# 정수, 실수, 문자열, 불리언
a = 10          # int
b = 3.14        # float
name = "Alice"  # str
flag = True     # bool
```

### 2‑2. 컬렉션(시퀀스) 자료

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser
print(type(chain))
print(chain)

<class 'langchain_core.runnables.base.RunnableSequence'>
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x13794dcd0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x140224090>, root_client=<openai.OpenAI object at 0x1378a6f90>, root_async_client=<openai.AsyncOpenAI object at 0x137e7d6d0>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')] last=StrOutputParser()


In [9]:
response = chain.invoke({"input":"지구의 자전주기는 얼마인가요?"})

In [10]:
print(type(response))
print(response)

<class 'str'>
지구의 자전주기는 24시간입니다. 이를 태양시라고 부르기도 합니다.
